<a href="https://colab.research.google.com/github/paulgureghian/Google_Colab_Notebooks/blob/master/Image_Classifier_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [1]:
### Mount Google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
### Install Pillow 
!pip install -q pillow==4.1.1

In [0]:
### Install Pytorch
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

In [0]:
### Import packages
import json
import torch
import PIL
import copy
import time
import numpy as np
from collections import OrderedDict
from torch import nn, optim
from torch.optim import lr_scheduler
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from torchvision import datasets, transforms, models
from google.colab import files
from PIL import Image

In [0]:
!pip install -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). You can [download the data here](https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip). The dataset is split into two parts, training and validation. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. If you use a pre-trained network, you'll also need to make sure the input data is resized to 224x224 pixels as required by the networks.

The validation set is used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks available from `torchvision` were trained on the ImageNet dataset where each color channel was normalized separately. For both sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.

In [0]:
### Load the data
data_dir = "/content/drive/My Drive/PyTorch Challenge/flower_data"
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'

In [0]:
### Define the transforms for the training and validation sets
training_transforms = transforms.Compose([transforms.RandomRotation(45),                                                                                                             
                                          transforms.RandomHorizontalFlip(),                                               
                                          transforms.RandomCrop(224),
                                          transforms.RandomResizedCrop(224), 
                                          transforms.ToTensor(),
                                          transforms.Normalize((0.485,0.456,0.406),
                                                               (0.229,0.224,0.225))])    
                                                                                                                                         
                                                                                                                         
validation_transforms = transforms.Compose([transforms.Resize(256), 
                                            transforms.CenterCrop(224), 
                                            transforms.ToTensor(),
                                            transforms.Normalize((0.485,0.456,0.406),
                                                                 (0.229,0.224,0.225))])
                                       
                                       
### Load the datasets with ImageFolder
training_dataset = datasets.ImageFolder(train_dir, transform =training_transforms)
validation_dataset = datasets.ImageFolder(valid_dir, transform =validation_transforms)

### Using the image datasets and the transforms, define the dataloaders
training_dataloader = torch.utils.data.DataLoader(training_dataset, batch_size =32, shuffle =True)
validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size =64)


### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [9]:
### Load label mapping file


cat_to_name = drive.CreateFile({'id' : '1zCjC-qnt9eoL6_U_cwgx-E-y869_rAPd'})
cat_to_name.GetContentFile('cat_to_name.json')
cat_to_name = json.load(open('cat_to_name.json'))
print(cat_to_name)


{'21': 'fire lily', '3': 'canterbury bells', '45': 'bolero deep blue', '1': 'pink primrose', '34': 'mexican aster', '27': 'prince of wales feathers', '7': 'moon orchid', '16': 'globe-flower', '25': 'grape hyacinth', '26': 'corn poppy', '79': 'toad lily', '39': 'siam tulip', '24': 'red ginger', '67': 'spring crocus', '35': 'alpine sea holly', '32': 'garden phlox', '10': 'globe thistle', '6': 'tiger lily', '93': 'ball moss', '33': 'love in the mist', '9': 'monkshood', '102': 'blackberry lily', '14': 'spear thistle', '19': 'balloon flower', '100': 'blanket flower', '13': 'king protea', '49': 'oxeye daisy', '15': 'yellow iris', '61': 'cautleya spicata', '31': 'carnation', '64': 'silverbush', '68': 'bearded iris', '63': 'black-eyed susan', '69': 'windflower', '62': 'japanese anemone', '20': 'giant white arum lily', '38': 'great masterwort', '4': 'sweet pea', '86': 'tree mallow', '101': 'trumpet creeper', '42': 'daffodil', '22': 'pincushion flower', '2': 'hard-leaved pocket orchid', '54': 's

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

model = models.vgg19(pretrained=True)

for param in model.parameters():
    param.requires_grad_(False)
    
model.classifier = nn.Sequential(OrderedDict([
                                ('fc1', nn.Linear(25088, 4096)),
                                ('relu1', nn.ReLU()),
                                ('dropout1', nn.Dropout(p=0.6)),
                                ('fc2', nn.Linear(4096, 2048)),
                                ('relu2', nn.ReLU()),
                                ('dropout2', nn.Dropout(p=0.6)),
                                ('fc3', nn.Linear(2048, 102)),
                                ('output', nn.LogSoftmax(dim=1)) 
                                ])) 
steps = 0
epochs = 1
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.2)

model.to(device) 

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inplace)

In [0]:
### Training loop
for epoch in range(epochs):
    scheduler.step()
    train_loss = 0
  
    model.train()
    train_losses = []
    for inputs, labels in training_dataloader:
        inputs, labels = inputs.to(device), labels.to(device) 
        
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
           
        loss = criterion(logps, labels)
    
        loss.backward()
                
        optimizer.step() 
 
        train_loss += loss.item()
           
    ### Validation loop 
    model.eval()
    accuracy = 0
    valid_loss = 0
    valid_losses = []
    for inputs, labels in validation_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
    
        output = model.forward(inputs)
    
        loss = criterion(output, labels)
    
        valid_loss += loss.item()
    
        ps = torch.exp(output)
        top_ps, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
       
    print("Epoch {}/{}.. ".format(epoch+1, epochs),
          "Training loss: {:.3f}.. ".format(train_loss / len(training_dataloader)),  
          "Validation loss: {:.3f}..".format(valid_loss / len(validation_dataloader)),
          "Validation accuracy: {:.3f}..".format(accuracy / len(validation_dataloader)))
   
    model.train()

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [0]:
### Save the checkpoint

model.epochs = epochs
model.acuracy = accuracy
model.train_loss = train_loss
model.valid_loss = valid_loss
model.cat_to_name = cat_to_name
model.class_to_idx = training_dataset.class_to_idx

model.scheduler = scheduler
model.optimizer = optimizer
model.criterion = criterion

checkpoint_path = "/content/drive/My Drive/Saved Checkpoint/checkpoint.pt"

model.to(device)
                  
checkpoint = {'model': model,
              'epochs': model.epochs,            
              'accuracy': model.accuracy,
              'train_loss': model.train_loss,
              'valid_loss': model.valid_loss,
              'cat_to_name': model.cat_to_name,
              'state_dict': model.state_dict(),            
              'scheduler': model.scheduler.state_dict(),
              'optimizer': model.optimizer.state_dict(),
              'criterion': model.criterion.state_dict(),                                              
              'class_to_idx': model.class_to_idx}
     
torch.save(checkpoint, checkpoint_path)                

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [0]:
### Load the checkpoint and rebuild the model
def load_checkpoint(checkpoint_path):
    
    checkpoint = torch.load(checkpoint_path)  
    
    model = checkpoint['model']
    model.epochs = checkpoint['epochs']
    model.accuracy = checkpoint['accuracy']
    model.train_loss = checkpoint['train_loss']
    model.valid_loss = checkpoint['valid_loss']    
    model.cat_to_name = checkpoint['cat_to_name']  
    model.load_state_dict(checkpoint['state_dict'])
    
    model.scheduler.load_state_dict(checkpoint['scheduler'])
    model.optimizer.load_state_dict(checkpoint['optimizer'])                       
    model.criterion.load_state_dict(checkpoint['criterion'])  
    
    model.class_to_idx = checkpoint['class_to_idx']    
    
    model.eval() 
    
    return model 

In [0]:
### Call 'load checkpoint()'
model = load_checkpoint(checkpoint_path)
print("Model:\n", model, '\n')
print("Model State Dict:\n", model.state_dict().keys(), '\n')  

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [0]:
### Image path
image_path = train_dir + '/1/image_06742.jpg'

In [0]:
### Image processing 
def process_image(image_path):
    
    ### Open image
    image = Image.open(image_path)
    
    image_transforms = transforms.Compose([transforms.Resize(256),
                                           transforms.CenterCrop(224), 
                                           transforms.ToTensor(),
                                           transforms.Normalize([0.485,0.456,0.406],
                                                                [0.229,0.224,0.225])])  
    
    image = image_transforms(image)
    image = np.array(image)
            
    print(image.shape) 
    
    return image

In [17]:
### Call 'process image()'
image = process_image(image)  

NameError: ignored

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [0]:
### Display the image
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    ax.grid(False) 
    
    return ax

In [0]:
### Call 'imshow()' 
imshow(image)  

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [0]:
### Define prediction function 
def predict(image, model, topk=5):
    
    model.eval() 
    model.to(device)   
      
    image = process_image(image)
    image_tensor = torch.from_numpy(image).type(torch.FloatTensor)
    
    image_input = image_tensor.unsqueeze(0)
          
    output = model(image_input)
    probabilities = torch.exp(output)
        
    probs = torch.topk(probabilities, topk) 
      
      
    classes = model.class_to_idx(probs)
    
    return probs, classes

In [0]:
probs, classes = predict(image, model) 

probs 
classes 

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the validation accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [0]:
# TODO: Display an image along with the top 5 classes

In [0]:
!pip freeze